<a href="https://colab.research.google.com/github/italolima04/data-science/blob/master/MapaEpidemiologico-COVID/Mapa_Epidemiol%C3%B3gico_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas, dados e Pluguins

In [290]:
#Baixando a biblioteca folium
pip install folium

In [0]:
#Importando as bibliotecas e pluguins que serão utilizados.
import pandas as pd
import numpy as np

import folium
from folium import plugins 

# Criando e Exibindo Mapa

In [0]:
#Criando uma instância do folium.Map() que será um map.
#Esse recebe alguns parâmetros, como largura, altura, coordenadas para o mapa e um nível de zoom inicial na exibição.
mapa = folium.Map(
    width='100%', height='100%',
    location=[-15.77972, -47.92972],
    zoom_start=4
)

In [293]:
#Exibir o mapa.
mapa

In [0]:
#Utilizando o método save p/ salvar o arquivo no formato HTML, possibilitando a visualização em uma página web.
mapa.save('mapa.html')

# Lendo e Visualizando Base de Dados

In [295]:
#Ler os dados que estão em formato CSV. Base de dados Brasil.io
df = pd.read_csv('https://brasil.io/dataset/covid19/caso?format=csv')
df

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-05-04,AC,Acrelândia,city,15,1,True,15256.0,1200013.0,98.32197,0.0667
1,2020-05-04,AC,Assis Brasil,city,1,0,True,7417.0,1200054.0,13.48254,0.0000
2,2020-05-04,AC,Bujari,city,2,0,True,10266.0,1200138.0,19.48178,0.0000
3,2020-05-04,AC,Cruzeiro do Sul,city,11,0,True,88376.0,1200203.0,12.44682,0.0000
4,2020-05-04,AC,Feijó,city,1,0,True,34780.0,1200302.0,2.87522,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
50797,2020-02-27,SP,NaN,state,1,0,False,45919049.0,35.0,0.00218,0.0000
50798,2020-02-26,SP,São Paulo,city,1,0,False,12252023.0,3550308.0,0.00816,0.0000
50799,2020-02-26,SP,NaN,state,1,0,False,45919049.0,35.0,0.00218,0.0000
50800,2020-02-25,SP,São Paulo,city,1,0,False,12252023.0,3550308.0,0.00816,0.0000


# Pré-processando os Dados

In [296]:
#Selecionar as colunas de interesse. Método loc do Pandas.
#Na coluna place_type selecionar somente as linhas que tem dados referentes aos estados.
estados = df.loc[df.place_type == 'state', ['state', 'confirmed', 'deaths', 'is_last']]
#Visualizar Data Frame
estados

,state,confirmed,deaths,is_last
14,AC,733,28,True
66,AL,1538,72,True
121,AM,7313,585,True
137,AP,1733,49,True
293,BA,3734,141,True
...,...,...,...,...
50793,SP,2,0,False
50795,SP,2,0,False
50797,SP,1,0,False
50799,SP,1,0,False


In [297]:
#Verificar se há dados faltando ou nulos.
estados.isnull().sum()

state        0
confirmed    0
deaths       0
is_last      0
dtype: int64

**Na data em que esse estudo foi feito, os dados estavam ok, mas e se fosse necessário um pré-processamento, o que poderia ser feito?**

1.   Remover os dados nulos
2.   Tratar esses dados 




**Remover os dados poderia nesse caso prejudicar o estudo, em questão de completude, concorda que se não houveram registros de mortes para aquele determinado estado, é mais eficiente substituir os valores ausentes por 0 ?**


Em casos de estudo da ciência de dados há outras maneiras, como preencher com a média, mediana e etc, isso depende do contexto e é uma importante decisão no pré-processamento dos dados.

In [0]:
#Uma forma de preencher com 0 os valores nulos
# estados = estados.loc[estados.deaths.isnull(), 'deaths'] = 0

**Existe já um método do Pandas que facilita esse trabalho**

In [0]:
# estados.fillna(0)

In [300]:
#Selecionar somente as últimas atualizações da base de dados para cada estado.
ultimos_dados = estados.loc[estados.is_last == True, :]
ultimos_dados

,state,confirmed,deaths,is_last
14,AC,733,28,True
66,AL,1538,72,True
121,AM,7313,585,True
137,AP,1733,49,True
293,BA,3734,141,True
373,ES,3351,123,True
437,GO,861,30,True
558,MA,4530,271,True
754,MG,2347,90,True
777,MS,274,10,True


In [301]:
#Verificar a quantidade de linhas do novo Data Frame (Compatível com o número de estados do Brasil)
len(ultimos_dados)

27

In [302]:
#Ao verificar que as últimas atualizações foram requisitadas, não é mais necessária a coluna 'is_last'
ultimos_dados = estados.loc[: , ['state', 'confirmed', 'deaths']]
ultimos_dados


,state,confirmed,deaths
14,AC,733,28
66,AL,1538,72
121,AM,7313,585
137,AP,1733,49
293,BA,3734,141
...,...,...,...
50793,SP,2,0
50795,SP,2,0
50797,SP,1,0
50799,SP,1,0


# Aplicando Logaritmo aos Dados

In [303]:
#Pegar o logaritmo dos dados e criar uma coluna extra para cada mortes e casos com seus respectivos logs
ultimos_dados['log_confirmed'] = np.log(ultimos_dados['confirmed']+1)
ultimos_dados['log_deaths'] = np.log(ultimos_dados['deaths']+1)
ultimos_dados.head(10)

,state,confirmed,deaths,log_confirmed,log_deaths
14,AC,733,28,6.598509,3.367296
66,AL,1538,72,7.338888,4.290459
121,AM,7313,585,8.897546,6.373320
137,AP,1733,49,7.458186,3.912023
293,BA,3734,141,8.225503,4.955827
373,ES,3351,123,8.117312,4.820282
437,GO,861,30,6.759255,3.433987
558,MA,4530,271,8.418698,5.605802
754,MG,2347,90,7.761319,4.510860
777,MS,274,10,5.616771,2.397895


# Importando Json para desenho do Mapa

In [0]:
 import json

In [0]:
#Arquivo Json com coordenadas p/ o desenho do Mapa.
br_estados = 'json.json'
geo_json_data = json.load(open(br_estados))

In [0]:
#geo_json_data

# Criando e Salvando Mapa de Casos

In [307]:
#Tipo de mapa mais simples.
mapa.choropleth(
    geo_data = geo_json_data,
    name = "Casos COVID",
    data = ultimos_dados,
    columns = ['state', 'log_confirmed'],
    key_on = 'feature.id',
    fill_color = "Reds",
    fill_opacity = 0.8,
    line_color = 'white',
    line_opacity = 0.8,
    show = True,
    legend_name = "Log de casos de COVID-19 por Estado no Brasil"
)

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [0]:
#Salvar em uma página Web.
mapa.save('mapa_casos.html')

# Criando e Salvando Mapas de Mortes

In [309]:
mapa.choropleth(
    geo_data = geo_json_data,
    name = "Mortes COVID",
    data = ultimos_dados,
    columns = ['state', 'log_deaths'],
    key_on = 'feature.id',
    fill_color = "YlOrRd",
    fill_opacity = 0.8,
    line_color = 'white',
    line_opacity = 0.8,
    show = True,
    legend_name = "Log de mortes de COVID-19 por Estado no Brasil"
)

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [0]:
#mapa

In [311]:
#Adicionar um Controler p/ definir qual dos dois mapas exibir
folium.LayerControl().add_to(mapa)

In [0]:
mapa.save('mapa_mortes.html')